In [ ]:
import json

color_columns = [
    'is_red', 'is_blue', 'is_black', 'is_gray',
    'is_orange', 'is_yellow', 'is_turquoise', 'is_green',
    'is_white', 'is_pink', 'is_brown', 'is_purple'
]

# Create a dictionary: { "0": "is_red", "1": "is_blue", ... }
color_label_map = {str(i): name for i, name in enumerate(color_columns)}

# Save to JSON
with open("color_label_map.json", "w") as f:
    json.dump(color_label_map, f, indent=4)

print("Saved label map to color_label_map.json")

In [5]:
import pandas as pd
import torch
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import numpy as np

# Define the color columns
color_columns = [
    'is_red', 'is_blue', 'is_black', 'is_gray',
    'is_orange', 'is_yellow', 'is_turquoise', 'is_green',
    'is_white', 'is_pink', 'is_brown', 'is_purple'
]

# --- Build the model architecture ---
def build_color_model(num_classes):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes),
        nn.Sigmoid()  # For multilabel probabilities
    )
    return model

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(color_columns)
color_model = build_color_model(num_classes)
color_model.load_state_dict(torch.load("D:/capstone/EZ-Rx-ID/models/color_model.pth", map_location=device))
color_model.to(device)
color_model.eval()

# Preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess the image
image_path = "D:/rximage/image/images/split_padded_rotated/00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1.jpg"
img = Image.open(image_path).convert("RGB")
input_tensor = preprocess(img).unsqueeze(0).to(device)

# Make prediction
with torch.no_grad():
    output = color_model(input_tensor)

# Convert to probability list
probabilities = output.cpu().numpy().flatten()

# Create DataFrame with column names and probabilities
color_df = pd.DataFrame([probabilities], columns=color_columns)

# View result
print(color_df)

C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


     is_red   is_blue  is_black   is_gray  is_orange  is_yellow  is_turquoise  \
0  0.213412  0.220393  0.164655  0.005882   0.185636   0.680384       0.00904   

   is_green  is_white  is_pink  is_brown  is_purple  
0  0.008783  0.907794  0.01683  0.313202    0.00979  


In [9]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd

# --- Build Scored Model ---
def build_scored_model():
    # --- Load model ---
    model = models.resnet18(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 1),
        nn.Sigmoid()
    )
    return model
# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scored_model = build_scored_model()
scored_model.load_state_dict(torch.load("D:/capstone/EZ-Rx-ID/models/scored_model.pth", map_location=device))
scored_model.to(device)
scored_model.eval()

# --- Preprocessing ---
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Inference ---
def predict_is_scored(image_path):
    # Load and preprocess image
    img = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        prob = scored_model(input_tensor).item()

    pred = int(prob >= 0.5)
    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    return pd.DataFrame([{
        "filename": base_filename,
        "is_scored_pred": pred,
        "is_scored_probability": prob
    }])

# Example usage:
image_path = "D:/rximage/image/images/split_padded_rotated/00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1.jpg"
scored_df = predict_is_scored(image_path)
print(scored_df)

C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


                                       filename  is_scored_pred  \
0  00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1               0   

   is_scored_probability  
0               0.000183  


In [15]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("merged_predictions.csv")

# Show the first few rows
df.columns

Index(['filename', 'is_scored_pred', 'is_scored_probability', 'is_red',
       'is_blue', 'is_black', 'is_gray', 'is_orange', 'is_yellow',
       'is_turquoise', 'is_green', 'is_white', 'is_pink', 'is_brown',
       'is_purple', 'shape_pred', 'is_printed_pred',
       'is_printed_pred_probability', 'base_filename'],
      dtype='object')

In [4]:
df.columns

Index(['filename', 'is_scored_pred', 'is_scored_probability', 'is_red',
       'is_blue', 'is_black', 'is_gray', 'is_orange', 'is_yellow',
       'is_turquoise', 'is_green', 'is_white', 'is_pink', 'is_brown',
       'is_purple', 'shape_pred', 'is_printed_pred',
       'is_printed_pred_probability', 'base_filename'],
      dtype='object')

In [ ]:
model = models.resnet18(pretrained=False)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)

In [11]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd

# --- Build printed Model ---
def build_printed_model():
    # --- Load model ---
    model = models.resnet18(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 1),
        nn.Sigmoid()
    )
    return model
# --- Load Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scored_model = build_printed_model()
scored_model.load_state_dict(torch.load("D:/capstone/EZ-Rx-ID/models/printed_model.pth", map_location=device))
scored_model.to(device)
scored_model.eval()

# --- Preprocessing ---
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Inference ---
def predict_is_printed(image_path):
    # Load and preprocess image
    img = Image.open(image_path).convert("RGB")
    input_tensor = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        prob = scored_model(input_tensor).item()

    pred = int(prob >= 0.5)
    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    return pd.DataFrame([{
        "filename": base_filename,
        "is_printed_pred": pred,
        "is_printed_probability": prob
    }])

# Example usage:
image_path = "D:/rximage/image/images/split_padded_rotated/00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1.jpg"
printed_df = predict_is_printed(image_path)
print(printed_df)

C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


                                       filename  is_printed_pred  \
0  00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1                1   

   is_printed_probability  
0                0.999966  


In [12]:
import os
import json
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd

# Paths
model_path = "D:/capstone/EZ-Rx-ID/models/shape_model.pth"
label_map_path = "D:/capstone/EZ-Rx-ID/backend/shape_model/shape_label_map.json"
image_path = "D:/rximage/image/images/split_padded_rotated/00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1.jpg"

# Load label map
with open(label_map_path, "r") as f:
    idx_to_label = json.load(f)

num_classes = len(idx_to_label)

# Load model
def build_shape_model(num_classes):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_shape_model(num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# Image transforms (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load and preprocess image
img = Image.open(image_path).convert("RGB")
input_tensor = transform(img).unsqueeze(0).to(device)

# Inference
with torch.no_grad():
    output = model(input_tensor)
    pred_idx = torch.argmax(output, dim=1).item()
    pred_label = idx_to_label[str(pred_idx)]

# Prepare result
filename = os.path.basename(image_path).replace(".jpg", "")
df = pd.DataFrame([{
    "filename": filename,
    "shape_pred": pred_label
}])

print(df)

C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


                                       filename shape_pred
0  00002-3228-30_RXNAVIMAGE10_391E1C80_bottom_1    CAPSULE


In [13]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cpu
False
